
# proyecto_01B_preprocesamiento_correcto

Notebook para preparar los datos y entrenar pipelines sin data leakage. Incluye limpieza determinística, split antes de transformaciones estadísticas y guardado del pipeline final.


In [25]:

import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    average_precision_score,
    precision_recall_curve,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    r2_score,
    mean_absolute_error,
    mean_squared_error,
)
from sklearn.base import clone
import joblib



## 1. Carga y limpieza determinística (sin fits)
- Eliminación de outliers evidentes (edad >120, ingresos = 666666)
- Cálculo de `ratio_compras_online` y eliminación de clientes sin compras
- Ingeniería básica de características determinísticas
- Exportación de un CSV limpio sin escalado


In [26]:

RAW_FILE = Path("data/interim/supermercado_features.csv")
CLEAN_FILE = Path("data/interim/supermercado_limpio.csv")

# Carga
df = pd.read_csv(RAW_FILE)
df["fecha_cliente"] = pd.to_datetime(df["fecha_cliente"])

# Reglas determinísticas
mask_edad = df["edad"] <= 120
mask_ingreso = df["ingresos"] != 666666
mask_basico = mask_edad & mask_ingreso

df = df[mask_basico].copy()

# Ratio de compras y clientes activos (evita inf y NaN)
df["ratio_compras_online"] = np.where(
    df["compras_totales"] > 0,
    df["compras_online"] / df["compras_totales"],
    np.nan,
)
df = df.dropna(subset=["ratio_compras_online"])

# Ingeniería básica
educ_map = {"Basica": 1, "Secundaria": 2, "Universitaria": 3, "Master": 4, "Doctorado": 5}
df["tiene_pareja"] = df["estado_civil"].isin(["Casado", "Union_Libre"]).astype(int)
df["educacion"] = df["educacion"].map(educ_map)
df["educacion_x_estado"] = df["educacion"] * df["tiene_pareja"]
df["hijos_casa"] = df["hijos_casa"] + df["adolescentes_casa"]
df["anio_alta"] = df["fecha_cliente"].dt.year

# Eliminar columnas redundantes
cols_drop = ["total_dependientes", "compras_online", "adolescentes_casa", "anio_nacimiento", "fecha_cliente"]
df = df.drop(columns=[c for c in cols_drop if c in df.columns])

# Guardar limpio sin escalado
CLEAN_FILE.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(CLEAN_FILE, index=False)
print(f"Dataset limpio guardado en {CLEAN_FILE} con forma {df.shape}")


Dataset limpio guardado en data/interim/supermercado_limpio.csv con forma (1982, 48)


> Nota de referencia de datos: `data/interim/supermercado_limpio.csv` es el insumo oficial para entrenamiento e inferencia (sin log1p ni escalado). El archivo `data/processed/supermercado_preprocesado.csv` del notebook 01 se mantiene solo para exploración/EDA con log1p aplicado; no debe usarse para servir modelos.



## 2. Split train/test antes de transformaciones
- One-hot encoding determinístico
- Estratificación por `respuesta`


In [27]:

# Separación de variables
X = df.drop("respuesta", axis=1)
y = df["respuesta"]

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
binary_cols = [
    c for c in X.columns
    if c not in categorical_cols and set(X[c].dropna().unique()).issubset({0, 1})
]
numeric_cols = [c for c in X.columns if c not in categorical_cols]
continuous_cols = [c for c in numeric_cols if c not in binary_cols]

# Split temporal: train antes de 2014, test 2014 en adelante
train_mask = df["anio_alta"] < 2014
X_train = X.loc[train_mask].copy()
X_test = X.loc[~train_mask].copy()
y_train = y.loc[train_mask].copy()
y_test = y.loc[~train_mask].copy()

print(f"Train: {X_train.shape}, Test: {X_test.shape}")


Train: (1508, 47), Test: (474, 47)


## 2.1 Chequeo de deriva temporal (train <2014 vs test ≥2014)

Comprobamos si cambian las distribuciones de variables clave entre train y test. Si el KS test muestra p-val < 0.05, hay indicios de deriva que pueden requerir recalibración o ajuste de umbral.


In [28]:
vars_drift = [
    "ingresos",
    "recencia",
    "ratio_compras_online",
    "tasa_compra_online",
]

vars_drift = [v for v in vars_drift if v in X_train.columns]

rows = []
for col in vars_drift:
    train_vals = X_train[col].dropna()
    test_vals = X_test[col].dropna()
    ks_stat, p_val = ks_2samp(train_vals, test_vals)
    rows.append(
        {
            "variable": col,
            "media_train": train_vals.mean(),
            "media_test": test_vals.mean(),
            "std_train": train_vals.std(),
            "std_test": test_vals.std(),
            "ks_stat": ks_stat,
            "p_valor": p_val,
            "deriva": p_val < 0.05,
        }
    )

chequeo_deriva = pd.DataFrame(rows)
chequeo_deriva


,variable,media_train,media_test,std_train,std_test,ks_stat,p_valor,deriva
0,ingresos,51289.062997,52254.506329,20406.423213,21433.321275,0.058375,0.163320,False
1,recencia,49.079576,48.046414,28.230594,29.627175,0.060765,0.132750,False
2,ratio_compras_online,0.413944,0.389195,0.144375,0.144908,0.112740,0.000187,True
3,tasa_compra_online,0.413983,0.389213,0.144370,0.144897,0.112740,0.000187,True



## 3. Pipeline de preprocesamiento
- Transformación log1p en continuas (opcional para modelos lineales)
- Estandarización solo en train
- PCA opcional (comentado por defecto)



## 3. Pipeline de preprocesamiento
- Transformación log1p en continuas (opcional para modelos lineales)
- Estandarización solo en train
- PCA opcional (comentado por defecto)


In [29]:

from sklearn.preprocessing import OneHotEncoder

categorical_encoder = OneHotEncoder(
    handle_unknown="ignore",
    drop="first",
    sparse_output=False,  # Cambiado de 'sparse' a 'sparse_output'
)

continuous_pipeline = Pipeline([
    ("log1p", FunctionTransformer(np.log1p, validate=False)),
    ("scaler", StandardScaler()),
    # ("pca", PCA(n_components=0.80))  # Activar si se requiere
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_encoder, categorical_cols),
        ("cont", continuous_pipeline, continuous_cols),
        ("bin", "passthrough", binary_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)



## 4. Entrenamiento y evaluación de clasificación sin leakage


In [30]:

# Clasificación: Logistic Regression con pesos {0:1, 1:7} y umbral fijo priorizando recall
clf = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(class_weight={0: 1, 1: 7}, max_iter=1000, random_state=42)),
])

# Split de validación (20% del train) para monitorear el umbral sin usar test
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

clf_val = clone(clf)
clf_val.fit(X_tr, y_tr)

y_proba_val = clf_val.predict_proba(X_val)[:, 1]
val_threshold = 0.35  # Umbral operativo elegido tras tuning enfocado en recall

y_pred_val = (y_proba_val >= val_threshold).astype(int)
val_precision = precision_score(y_val, y_pred_val)
val_recall = recall_score(y_val, y_pred_val)
val_f1 = f1_score(y_val, y_pred_val)

# Entrenamiento final en todo el train
clf = clone(clf)
clf.fit(X_train, y_train)

y_proba_train = clf.predict_proba(X_train)[:, 1]
y_proba_test = clf.predict_proba(X_test)[:, 1]

a = roc_auc_score(y_train, y_proba_train)
b = roc_auc_score(y_test, y_proba_test)
auprc = average_precision_score(y_test, y_proba_test)

cv_scores_auc = cross_val_score(clf, X_train, y_train, cv=5, scoring="roc_auc")

print(f"Umbral fijo para inferencia: {val_threshold:.3f} (prioridad al recall)")
print(f"Validación (20% train) -> Precision: {val_precision:.3f} | Recall: {val_recall:.3f} | F1: {val_f1:.3f}")
print(f"AUC Train: {a:.4f} | AUC Test: {b:.4f} | Gap: {a-b:.4f}")
print(f"AUCPR Test : {auprc:.4f}")
print(f"CV AUC (5-fold): {cv_scores_auc.mean():.4f} ± {cv_scores_auc.std():.4f}")

# Reporte con umbral fijado

y_pred_test_opt = (y_proba_test >= val_threshold).astype(int)
acc_test = accuracy_score(y_test, y_pred_test_opt)

print("Classification report (test, umbral fijo de inferencia):")
print(classification_report(y_test, y_pred_test_opt))
print(f"Accuracy test: {acc_test:.4f}")


Umbral fijo para inferencia: 0.350 (prioridad al recall)
Validación (20% train) -> Precision: 0.420 | Recall: 0.894 | F1: 0.571
AUC Train: 0.9240 | AUC Test: 0.9535 | Gap: -0.0295
AUCPR Test : 0.6061
CV AUC (5-fold): 0.8949 ± 0.0182
Classification report (test, umbral fijo de inferencia):
              precision    recall  f1-score   support

           0       0.98      0.92      0.95       431
           1       0.50      0.81      0.62        43

    accuracy                           0.91       474
   macro avg       0.74      0.87      0.78       474
weighted avg       0.94      0.91      0.92       474

Accuracy test: 0.9093



### Interpretación clasificación (sin leakage, split temporal)
- Modelo final: LogisticRegression con class_weight={0:1, 1:7} para penalizar más los falsos negativos.
- Umbral operativo fijo 0.35 priorizando recall (seleccionado por tuning externo); se monitorea en una validación 20% del train para evitar fuga.
- AUC train/test y AUCPR se mantienen altos; el gap es controlado y se valida estabilidad con CV AUC 5-fold.
- Este pipeline y umbral son la referencia para producción; cualquier ajuste futuro debe recalcular métricas en el corte temporal.



## 5. Entrenamiento y evaluación de regresión sin leakage
Usa el mismo preprocesador; ajusta el objetivo `gasto_total` en log1p.


### Interpretación regresión (sin fuga, split temporal)
- R² train/test: 0.939 / 0.867 con MAE ≈ 127 y RMSE ≈ 206: métricas más realistas tras eliminar features derivadas del gasto.
- El corte temporal (<2014 / 2014+) evita fuga por tiempo; aún hay error alto, sugiere modelos más robustos o features adicionales no fugas (p.ej., ingresos, recencia, ratio_compras_online).
- Se mantiene log1p+escalado en train vía pipeline para consistencia.


In [31]:

# Preparar datos de regresión sin fuga y con corte temporal
leak_cols = [
    "gasto_vinos", "gasto_frutas", "gasto_carnes", "gasto_pescado", "gasto_dulces", "gasto_oro",
    "gasto_promedio", "prop_gasto_vinos", "prop_gasto_frutas", "prop_gasto_carnes", "prop_gasto_pescado",
    "prop_gasto_dulces", "prop_gasto_oro", "ticket_promedio", "gasto_x_recencia"
]
y_reg = np.log1p(df["gasto_total"])
X_reg_raw = df.drop(["respuesta", "gasto_total"], axis=1)
X_reg_raw = X_reg_raw.drop(columns=[c for c in leak_cols if c in X_reg_raw.columns])

categorical_cols_reg = X_reg_raw.select_dtypes(include=["object"]).columns.tolist()
binary_cols_reg = [
    c for c in X_reg_raw.columns
    if c not in categorical_cols_reg and set(X_reg_raw[c].dropna().unique()).issubset({0, 1})
]
numeric_cols_reg = [c for c in X_reg_raw.columns if c not in categorical_cols_reg]
continuous_cols_reg = [c for c in numeric_cols_reg if c not in binary_cols_reg]

train_mask_reg = df["anio_alta"] < 2014
Xr_train = X_reg_raw.loc[train_mask_reg].copy()
Xr_test = X_reg_raw.loc[~train_mask_reg].copy()
yr_train = y_reg.loc[train_mask_reg].copy()
yr_test = y_reg.loc[~train_mask_reg].copy()

# Cambia el parámetro 'sparse' a 'sparse_output' para compatibilidad con versiones recientes de scikit-learn.
categorical_encoder_reg = OneHotEncoder(
    handle_unknown="ignore",
    drop="first",
    sparse_output=False,
)

continuous_pipeline_reg = Pipeline([
    ("log1p", FunctionTransformer(np.log1p, validate=False)),
    ("scaler", StandardScaler()),
])

preprocessor_reg = ColumnTransformer(
    transformers=[
        ("cat", categorical_encoder_reg, categorical_cols_reg),
        ("cont", continuous_pipeline_reg, continuous_cols_reg),
        ("bin", "passthrough", binary_cols_reg),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

reg = Pipeline([
    ("preprocessor", preprocessor_reg),
    ("model", GradientBoostingRegressor(random_state=42))
])
reg.fit(Xr_train, yr_train)

pred_train_log = reg.predict(Xr_train)
pred_test_log = reg.predict(Xr_test)

# Convertir predicciones y objetivos a escala original para métricas interpretables
pred_train = np.expm1(pred_train_log)
pred_test = np.expm1(pred_test_log)
yr_train_orig = np.expm1(yr_train)
yr_test_orig = np.expm1(yr_test)

print(f"R2 train: {r2_score(yr_train_orig, pred_train):.4f}")
print(f"R2 test : {r2_score(yr_test_orig, pred_test):.4f}")
print(f"MAE test: {mean_absolute_error(yr_test_orig, pred_test):.4f}")
print(f"RMSE test: {np.sqrt(mean_squared_error(yr_test_orig, pred_test)):.4f}")


R2 train: 0.9102
R2 test : 0.8926
MAE test: 95.4121
RMSE test: 185.3552



## 6. Guardado de pipelines y metadatos


In [32]:

models_dir = Path("models")
models_dir.mkdir(parents=True, exist_ok=True)

joblib.dump(clf, models_dir / "pipeline_clasificacion_sin_leakage.pkl")
joblib.dump(reg, models_dir / "pipeline_regresion_sin_leakage.pkl")

# Usar los preprocesadores ya entrenados dentro de los pipelines
preproc_clf = clf.named_steps["preprocessor"]
preproc_reg = reg.named_steps["preprocessor"]

# Extraer nombres de features sin depender de get_feature_names_out en pipelines con FunctionTransformer
cat_names = preproc_clf.named_transformers_["cat"].get_feature_names_out(categorical_cols)
cont_names = continuous_cols
bin_names = binary_cols
feature_names_input = list(cat_names) + cont_names + bin_names

cat_names_reg = preproc_reg.named_transformers_["cat"].get_feature_names_out(categorical_cols_reg)
cont_names_reg = continuous_cols_reg
bin_names_reg = binary_cols_reg
reg_feature_names_input = list(cat_names_reg) + cont_names_reg + bin_names_reg

# Métricas de clasificación (umbral fijo)
clf_y_pred_opt = (y_proba_test >= val_threshold).astype(int)
clf_precision = precision_score(y_test, clf_y_pred_opt)
clf_recall = recall_score(y_test, clf_y_pred_opt)
clf_f1 = f1_score(y_test, clf_y_pred_opt)
clf_accuracy = accuracy_score(y_test, clf_y_pred_opt)
auc_train = float(a)
auc_test = float(b)
aucpr_test = float(auprc)
cv_auc_mean = float(cv_scores_auc.mean())
cv_auc_std = float(cv_scores_auc.std())

# Métricas de regresión (escala original)
r2_train = r2_score(yr_train_orig, pred_train)
r2_test = r2_score(yr_test_orig, pred_test)
mae_reg = mean_absolute_error(yr_test_orig, pred_test)
rmse_reg = np.sqrt(mean_squared_error(yr_test_orig, pred_test))

metadata = {
    "feature_names_input": feature_names_input,
    "raw_feature_names": X_train.columns.tolist(),
    "continuous_cols": continuous_cols,
    "binary_cols": binary_cols,
    "categorical_original": categorical_cols,
    "train_shape": X_train.shape,
    "test_shape": X_test.shape,
    "reg_feature_names_input": reg_feature_names_input,
    "reg_raw_feature_names": Xr_train.columns.tolist(),
    "reg_continuous_cols": continuous_cols_reg,
    "reg_binary_cols": binary_cols_reg,
    "categorical_cols_reg": categorical_cols_reg,
    "temporal_split": {"train_years": "<2014", "test_years": "2014+"},
    "clf_threshold": float(val_threshold),
    "metrics": {
        "auc_train": auc_train,
        "auc_test": auc_test,
        "aucpr_test": aucpr_test
    },
    "clf_config": {
        "model": "LogisticRegression",
        "class_weight": {0: 1, 1: 7},
        "threshold": float(val_threshold),
        "optimized_for": "recall_min_60_percent",
    },
    "clf_metrics": {
        "threshold_opt": float(val_threshold),
        "auc_train": auc_train,
        "auc_test": auc_test,
        "aucpr_test": aucpr_test,
        "precision_test": float(clf_precision),
        "recall_test": float(clf_recall),
        "f1_test": float(clf_f1),
        "accuracy_test": float(clf_accuracy),
        "cv_auc_mean": cv_auc_mean,
        "cv_auc_std": cv_auc_std,
    },
    "reg_metrics": {
        "r2_train": float(r2_train),
        "r2_test": float(r2_test),
        "mae_test": float(mae_reg),
        "rmse_test": float(rmse_reg)
    }
}
joblib.dump(metadata, models_dir / "pipeline_metadata.pkl")
print("Pipelines y metadatos guardados en models/")


Pipelines y metadatos guardados en models/


## REGRESIÓN (Predicción de Gasto)

### Modelo Final
**Gradient Boosting Regressor**

### Métricas de Desempeño (Escala Original en Euros)
| Métrica | Train | Test | Interpretación |
|---------|-------|------|----------------|
| **R²** | 0.9390 | **0.8670** | Explica 86.7% de varianza en test |
| **MAE** | - | **127.44€** | Error promedio de predicción |
| **RMSE** | - | **206.47€** | Error cuadrático medio |

### Análisis de Overfitting
- **Gap R²**: 0.9390 - 0.8670 = **0.072** (7.2%)
- **Estado**: Ligero overfitting, pero dentro de rango aceptable
- El modelo generaliza bien a datos no vistos

### Interpretación de Negocio
- **MAE 127€**: En promedio, las predicciones se desvían ±127€ del gasto real
- **RMSE 206€**: Errores grandes (outliers) son penalizados más fuertemente
- Para un cliente con gasto promedio de ~600€, esto representa un **error relativo del 21%**

### Variables Clave Eliminadas (Sin Leakage)
Se excluyeron correctamente variables derivadas del gasto:
- `gasto_vinos`, `gasto_frutas`, `gasto_carnes`, etc.
- `gasto_promedio`, `prop_gasto_*`
- `ticket_promedio`, `gasto_x_recencia`
